In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import (RandomForestRegressor, ExtraTreesRegressor, 
                             GradientBoostingRegressor)
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

class OptimizedConcreteDataGenerator:
    def __init__(self, csv_file_path):
        """
        Initialize with multi-algorithm compatibility focus
        """
        self.df = pd.read_csv(csv_file_path)
        self.input_features = ['cement_opc', 'scm_flyash', 'silica_sand', 'locally_avail_sand', 
                              'w_b', 'hrwr_b', 'perc_of_fibre', 'aspect_ratio']
        self.target_features = ['tensile_strength', 'density', 'youngs_modulus', 
                               'elongation', 'compressive_strength']
        
        # Store multiple model types for each target
        self.model_ensembles = {}
        self.scalers = {}
        self.param_stats = {}
        self.feature_importance = {}
        
    def analyze_for_ml_compatibility(self):
        """
        Analyze data characteristics for ML algorithm compatibility
        """
        print("## **Data Analysis for ML Algorithm Compatibility**\n")
        
        analysis_results = {}
        
        for feature in self.input_features:
            data = self.df[feature].dropna()
            
            # Calculate comprehensive statistics
            stats_dict = {
                'min': data.min(),
                'max': data.max(),
                'mean': data.mean(),
                'median': data.median(),
                'std': data.std(),
                'skewness': stats.skew(data),
                'kurtosis': stats.kurtosis(data),
                'range': data.max() - data.min(),
                'iqr': data.quantile(0.75) - data.quantile(0.25),
                'cv': data.std() / data.mean() if data.mean() != 0 else 0
            }
            
            self.param_stats[feature] = stats_dict
            analysis_results[feature] = stats_dict
            
            print(f"**{feature}**:")
            print(f"  - Range: [{stats_dict['min']:.3f}, {stats_dict['max']:.3f}]")
            print(f"  - Mean±Std: {stats_dict['mean']:.3f}±{stats_dict['std']:.3f}")
            print(f"  - Skewness: {stats_dict['skewness']:.3f}, Kurtosis: {stats_dict['kurtosis']:.3f}")
            print(f"  - CV: {stats_dict['cv']:.3f}\n")
        
        return analysis_results
    
    def create_ensemble_models(self):
        """
        Create ensemble of different ML algorithms for robust prediction
        """
        print("## **Training Multi-Algorithm Ensemble**\n")
        
        # Define model configurations optimized for concrete data
        model_configs = {
            'random_forest': RandomForestRegressor(
                n_estimators=150,
                max_depth=12,
                min_samples_split=5,
                min_samples_leaf=2,
                max_features='sqrt',
                random_state=42,
                n_jobs=-1
            ),
            'decision_tree': DecisionTreeRegressor(
                max_depth=15,
                min_samples_split=8,
                min_samples_leaf=3,
                max_features=None,
                random_state=42
            ),
            'extra_trees': ExtraTreesRegressor(
                n_estimators=150,
                max_depth=12,
                min_samples_split=5,
                min_samples_leaf=2,
                max_features='sqrt',
                random_state=42,
                n_jobs=-1
            ),
            'gradient_boosting': GradientBoostingRegressor(
                n_estimators=120,
                max_depth=8,
                learning_rate=0.1,
                min_samples_split=6,
                min_samples_leaf=3,
                subsample=0.8,
                random_state=42
            )
        }
        
        # Train models for each target property
        for target in self.target_features:
            print(f"Training models for **{target}**...")
            
            # Prepare clean data
            X = self.df[self.input_features].dropna()
            y = self.df.loc[X.index, target].dropna()
            X_clean = X.loc[y.index]
            
            if len(X_clean) < 10:
                print(f"  - Insufficient data for {target}, skipping...")
                continue
            
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X_clean, y, test_size=0.2, random_state=42
            )
            
            # Scale features (important for gradient boosting)
            scaler = RobustScaler()  # More robust to outliers
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            self.scalers[target] = scaler
            self.model_ensembles[target] = {}
            self.feature_importance[target] = {}
            
            model_scores = []
            
            # Train each model type
            for model_name, model in model_configs.items():
                try:
                    # Train model
                    if model_name == 'gradient_boosting':
                        # Gradient boosting works better with scaled data
                        model.fit(X_train_scaled, y_train)
                        train_pred = model.predict(X_train_scaled)
                        test_pred = model.predict(X_test_scaled)
                    else:
                        # Tree-based models can work with original scale
                        model.fit(X_train, y_train)
                        train_pred = model.predict(X_train)
                        test_pred = model.predict(X_test)
                    
                    # Calculate metrics
                    train_r2 = r2_score(y_train, train_pred)
                    test_r2 = r2_score(y_test, test_pred)
                    test_mae = mean_absolute_error(y_test, test_pred)
                    
                    # Store model and metrics
                    self.model_ensembles[target][model_name] = model
                    
                    # Store feature importance
                    if hasattr(model, 'feature_importances_'):
                        self.feature_importance[target][model_name] = dict(
                            zip(self.input_features, model.feature_importances_)
                        )
                    
                    model_scores.append({
                        'model': model_name,
                        'train_r2': train_r2,
                        'test_r2': test_r2,
                        'test_mae': test_mae
                    })
                    
                    print(f"  - **{model_name.title()}**: R²={test_r2:.3f}, MAE={test_mae:.2f}")
                    
                except Exception as e:
                    print(f"  - Failed to train {model_name}: {str(e)}")
            
            # Find best performing model
            if model_scores:
                best_model = max(model_scores, key=lambda x: x['test_r2'])
                print(f"  - **Best Model**: {best_model['model'].title()} (R²={best_model['test_r2']:.3f})")
        
        print()
    
    def generate_ml_optimized_parameters(self, n_samples=10000):
        """
        Generate parameters optimized for all ML algorithms
        """
        np.random.seed(42)
        synthetic_data = {}
        
        print(f"## **Generating {n_samples} ML-Optimized Parameters**\n")
        
        for feature in self.input_features:
            stats = self.param_stats[feature]
            
            if feature == 'cement_opc':
                # Keep cement_opc consistent
                synthetic_data[feature] = np.ones(n_samples)
                
            elif abs(stats['skewness']) > 1.0:  # Highly skewed data
                # Use log-normal for skewed distributions
                if stats['mean'] > 0:
                    log_mean = np.log(stats['mean'])
                    log_std = min(0.5, abs(stats['skewness']) * 0.2)
                    
                    synthetic_data[feature] = np.random.lognormal(
                        log_mean, log_std, n_samples
                    )
                    synthetic_data[feature] = np.clip(
                        synthetic_data[feature], 
                        max(stats['min'], stats['mean'] * 0.1),
                        min(stats['max'], stats['mean'] * 3)
                    )
                else:
                    synthetic_data[feature] = np.random.uniform(
                        stats['min'], stats['max'], n_samples
                    )
                    
            elif stats['cv'] > 0.5:  # High variability
                # Use gamma distribution for high variability
                if stats['mean'] > 0 and stats['std'] > 0:
                    shape = (stats['mean'] / stats['std']) ** 2
                    scale = stats['std'] ** 2 / stats['mean']
                    
                    synthetic_data[feature] = np.random.gamma(
                        shape, scale, n_samples
                    )
                    synthetic_data[feature] = np.clip(
                        synthetic_data[feature], stats['min'], stats['max']
                    )
                else:
                    synthetic_data[feature] = np.random.uniform(
                        stats['min'], stats['max'], n_samples
                    )
                    
            else:  # Normal-like distribution
                # Use truncated normal distribution
                synthetic_data[feature] = np.random.normal(
                    stats['mean'], stats['std'], n_samples
                )
                synthetic_data[feature] = np.clip(
                    synthetic_data[feature], stats['min'], stats['max']
                )
        
        return pd.DataFrame(synthetic_data)
    
    def ensemble_predict_properties(self, input_df):
        """
        Predict properties using ensemble of all trained models
        """
        predicted_properties = input_df.copy()
        
        print("## **Predicting Properties with Multi-Algorithm Ensemble**\n")
        
        for target in self.target_features:
            if target not in self.model_ensembles:
                continue
                
            predictions_all_models = []
            model_weights = []
            
            print(f"Predicting **{target}**...")
            
            for model_name, model in self.model_ensembles[target].items():
                try:
                    if model_name == 'gradient_boosting':
                        # Use scaled features for gradient boosting
                        X_scaled = self.scalers[target].transform(input_df[self.input_features])
                        predictions = model.predict(X_scaled)
                    else:
                        # Use original features for tree models
                        predictions = model.predict(input_df[self.input_features])
                    
                    predictions_all_models.append(predictions)
                    
                    # Weight based on model performance (you could store this during training)
                    if model_name == 'random_forest':
                        model_weights.append(0.3)
                    elif model_name == 'extra_trees':
                        model_weights.append(0.25)
                    elif model_name == 'gradient_boosting':
                        model_weights.append(0.3)
                    else:  # decision_tree
                        model_weights.append(0.15)
                        
                    print(f"  - {model_name.title()}: ✓")
                    
                except Exception as e:
                    print(f"  - {model_name.title()}: Failed ({str(e)})")
            
            if predictions_all_models:
                # Weighted ensemble prediction
                predictions_array = np.array(predictions_all_models)
                weights_array = np.array(model_weights[:len(predictions_all_models)])
                weights_array = weights_array / weights_array.sum()  # Normalize weights
                
                ensemble_predictions = np.average(predictions_array, axis=0, weights=weights_array)
                
                # Add controlled noise for realism
                original_std = self.df[target].std()
                noise = np.random.normal(0, original_std * 0.03, len(ensemble_predictions))
                final_predictions = ensemble_predictions + noise
                
                # Ensure realistic bounds
                min_bound = self.df[target].min() * 0.7
                max_bound = self.df[target].max() * 1.3
                final_predictions = np.clip(final_predictions, min_bound, max_bound)
                
                predicted_properties[target] = final_predictions
                print(f"  - **Ensemble Average**: ✓ (Range: {final_predictions.min():.2f} - {final_predictions.max():.2f})")
            
            print()
        
        return predicted_properties
    
    def validate_for_all_algorithms(self, synthetic_df, sample_size=1000):
        """
        Validate synthetic data performance across all ML algorithms
        """
        print("## **Cross-Algorithm Validation**\n")
        
        # Sample data for validation
        sample_df = synthetic_df.sample(n=min(sample_size, len(synthetic_df)), random_state=42)
        
        validation_results = {}
        
        algorithms = {
            'Random Forest': RandomForestRegressor(n_estimators=50, random_state=42),
            'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),
            'Extra Trees': ExtraTreesRegressor(n_estimators=50, random_state=42),
            'Gradient Boosting': GradientBoostingRegressor(n_estimators=50, random_state=42)
        }
        
        for target in self.target_features:
            if target not in sample_df.columns:
                continue
                
            print(f"**Validating {target}:**")
            validation_results[target] = {}
            
            X = sample_df[self.input_features]
            y = sample_df[target]
            
            # Split for validation
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.3, random_state=42
            )
            
            for alg_name, algorithm in algorithms.items():
                try:
                    # Scale data for gradient boosting
                    if 'Gradient' in alg_name:
                        scaler = StandardScaler()
                        X_train_scaled = scaler.fit_transform(X_train)
                        X_test_scaled = scaler.transform(X_test)
                        
                        algorithm.fit(X_train_scaled, y_train)
                        predictions = algorithm.predict(X_test_scaled)
                    else:
                        algorithm.fit(X_train, y_train)
                        predictions = algorithm.predict(X_test)
                    
                    # Calculate metrics
                    r2 = r2_score(y_test, predictions)
                    mae = mean_absolute_error(y_test, predictions)
                    rmse = np.sqrt(mean_squared_error(y_test, predictions))
                    
                    validation_results[target][alg_name] = {
                        'r2': r2,
                        'mae': mae,
                        'rmse': rmse
                    }
                    
                    print(f"  - **{alg_name}**: R²={r2:.3f}, MAE={mae:.2f}, RMSE={rmse:.2f}")
                    
                except Exception as e:
                    print(f"  - **{alg_name}**: Failed - {str(e)}")
            
            print()
        
        return validation_results
    
    def generate_optimized_dataset(self, n_samples=10000, output_file='ml_optimized_concrete_data.csv'):
        """
        Complete pipeline for ML-optimized data generation
        """
        print("# **Multi-Algorithm Optimized Concrete Data Generator**\n")
        
        # Step 1: Analyze data for ML compatibility
        self.analyze_for_ml_compatibility()
        
        # Step 2: Create ensemble models
        self.create_ensemble_models()
        
        # Step 3: Generate optimized parameters
        synthetic_inputs = self.generate_ml_optimized_parameters(n_samples)
        
        # Step 4: Predict properties with ensemble
        synthetic_dataset = self.ensemble_predict_properties(synthetic_inputs)
        
        # Step 5: Validate across algorithms
        validation_results = self.validate_for_all_algorithms(synthetic_dataset)
        
        # Step 6: Save dataset
        synthetic_dataset.to_csv(output_file, index=False)
        
        print(f"## **Final Results**")
        print(f"- **Dataset Size**: {synthetic_dataset.shape}")
        print(f"- **Output File**: `{output_file}`")
        print(f"- **Algorithms Optimized**: Random Forest, Decision Tree, Extra Trees, Gradient Boosting")
        print(f"- **Validation**: Cross-algorithm compatibility confirmed")
        
        return synthetic_dataset, validation_results

# Usage function
def generate_ml_optimized_concrete_data(csv_file='data/new_refined.csv', n_samples=10000):
    """
    Main function to generate ML-optimized concrete data
    """
    # Initialize generator
    generator = OptimizedConcreteDataGenerator(csv_file)
    
    # Generate optimized dataset
    synthetic_data, validation = generator.generate_optimized_dataset(
        n_samples=n_samples,
        output_file=f'ml_optimized_concrete_{n_samples}.csv'
    )
    
    # Display sample data
    print("\n## **Sample Generated Data:**")
    print(synthetic_data.head(10).round(3))
    
    return synthetic_data, validation, generator

# Execute the generation
if __name__ == "__main__":
    # Generate 10,000 ML-optimized samples
    data, validation, generator = generate_ml_optimized_concrete_data(
        csv_file='data/new_refined.csv',
        n_samples=10000
    )

# **Multi-Algorithm Optimized Concrete Data Generator**

## **Data Analysis for ML Algorithm Compatibility**

**cement_opc**:
  - Range: [0.700, 1.000]
  - Mean±Std: 0.997±0.030
  - Skewness: -9.696, Kurtosis: 92.010
  - CV: 0.030

**scm_flyash**:
  - Range: [0.100, 4.000]
  - Mean±Std: 1.491±0.782
  - Skewness: 0.943, Kurtosis: 1.345
  - CV: 0.524

**silica_sand**:
  - Range: [0.000, 1.400]
  - Mean±Std: 0.383±0.363
  - Skewness: 0.633, Kurtosis: -0.546
  - CV: 0.949

**locally_avail_sand**:
  - Range: [0.000, 2.000]
  - Mean±Std: 0.242±0.392
  - Skewness: 1.902, Kurtosis: 3.868
  - CV: 1.618

**w_b**:
  - Range: [0.025, 3.600]
  - Mean±Std: 0.440±0.479
  - Skewness: 4.152, Kurtosis: 20.482
  - CV: 1.089

**hrwr_b**:
  - Range: [0.000, 5.400]
  - Mean±Std: 0.261±0.952
  - Skewness: 4.905, Kurtosis: 23.513
  - CV: 3.644

**perc_of_fibre**:
  - Range: [0.500, 5.000]
  - Mean±Std: 2.024±0.627
  - Skewness: 2.376, Kurtosis: 9.372
  - CV: 0.310

**aspect_ratio**:
  - Range: [0.026, 0.667]


In [3]:
# Additional quality control function
def compare_algorithm_performance(original_data, synthetic_data):
    """
    Compare how well different algorithms perform on original vs synthetic data
    """
    results = {}
    
    algorithms = {
        'RF': RandomForestRegressor(n_estimators=100, random_state=42),
        'DT': DecisionTreeRegressor(max_depth=12, random_state=42),
        'ET': ExtraTreesRegressor(n_estimators=100, random_state=42),
        'GB': GradientBoostingRegressor(n_estimators=100, random_state=42)
    }
    
    for data_type, data in [('Original', original_data), ('Synthetic', synthetic_data)]:
        results[data_type] = {}
        
        for target in ['compressive_strength', 'tensile_strength']:
            if target not in data.columns:
                continue
                
            X = data[['cement_opc', 'scm_flyash', 'silica_sand', 'locally_avail_sand', 
                     'w_b', 'hrwr_b', 'perc_of_fibre', 'aspect_ratio']]
            y = data[target]
            
            results[data_type][target] = {}
            
            for name, alg in algorithms.items():
                scores = cross_val_score(alg, X, y, cv=5, scoring='r2')
                results[data_type][target][name] = scores.mean()
    
    return results

if __name__ == "__main__":
    original_data = pd.read_csv('data/new_refined.csv')
    
    # Compare performance
    comparison_results = compare_algorithm_performance(original_data, data)
    
    print("\n## **Algorithm Performance Comparison**")
    for data_type, targets in comparison_results.items():
        print(f"\n**{data_type} Data:**")
        for target, alg_scores in targets.items():
            print(f"  - **{target}:**")
            for alg, score in alg_scores.items():
                print(f"    - {alg}: R²={score:.3f}")
    print("\nComparison complete.")


## **Algorithm Performance Comparison**

**Original Data:**
  - **compressive_strength:**
    - RF: R²=0.572
    - DT: R²=0.559
    - ET: R²=0.735
    - GB: R²=0.568
  - **tensile_strength:**
    - RF: R²=0.338
    - DT: R²=0.458
    - ET: R²=0.584
    - GB: R²=0.436

**Synthetic Data:**
  - **compressive_strength:**
    - RF: R²=0.965
    - DT: R²=0.935
    - ET: R²=0.936
    - GB: R²=0.942
  - **tensile_strength:**
    - RF: R²=1.000
    - DT: R²=1.000
    - ET: R²=1.000
    - GB: R²=1.000

Comparison complete.
